## Google Gemini API tests

In [173]:
from dotenv import load_dotenv
load_dotenv()

from google import genai
from google.genai import types

from pydantic import BaseModel, Field
from typing import List
import json

In [2]:
client = genai.Client()

In [4]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Tell me a corny joke"
)
print(response.text)

Why don't scientists trust atoms?

Because they make up everything!



Check Async

In [6]:
response = await client.aio.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Tell me a corny joke"
)
print(response.text)

Why don't eggs tell jokes? 

Because they'd crack each other up!



### Explore API features

List models

In [40]:
models = [i for i in await client.aio.models.list()]

In [48]:
[i for i in models if '2.5' in i.name]

[Model(name='models/gemini-2.5-pro-exp-03-25', display_name='Gemini 2.5 Pro Experimental 03-25', description='Experimental release (March 25th, 2025) of Gemini 2.5 Pro', version='2.5-exp-03-25', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1048576, output_token_limit=65536, supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gemini-2.5-pro-preview-03-25', display_name='Gemini 2.5 Pro Preview 03-25', description='Gemini 2.5 Pro Preview 03-25', version='2.5-preview-03-25', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1048576, output_token_limit=65536, supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'], default_checkpoint_id=None, checkpoints=None),
 Mod

Base settings

In [102]:
model = 'models/gemini-2.5-flash-preview-05-20'
#model = 'models/gemini-2.5-flash-preview-04-17-thinking'
#model = 'models/gemini-2.5-pro-exp-03-25'

In [103]:
system_instruction = 'You are a very funny teaching assistant. You are given a question and you need to answer it in a way that is easy to understand and funny.'

user_instruction = 'Think thoroughly and in 3 sentences explain how rain is formed'

In [153]:
response = await client.aio.models.generate_content(
    model=model, 
    contents=user_instruction,
    config=types.GenerateContentConfig(
        system_instruction = system_instruction,
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            thinking_budget=4096
            ),
        temperature=0.3,
        candidate_count=1,
        response_mime_type='text/plain', #application/json,
        response_schema= None,
        tools = None,
        tool_config = None,
    )
)

In [154]:
print(f'Prompt tokens: {response.usage_metadata.prompt_token_count}')
print(f'Thoughts tokens: {response.usage_metadata.thoughts_token_count}')
print(f'Candidates tokens: {response.usage_metadata.candidates_token_count}')

print(f'Total tokens: {response.usage_metadata.total_token_count}')

Prompt tokens: 44
Thoughts tokens: 1013
Candidates tokens: 131
Total tokens: 1188


In [155]:
response.candidates[0].content.parts

[Part(video_metadata=None, thought=True, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='**Understanding Rain: A Lighthearted Explanation**\n\nOkay, here\'s how I\'d explain rain: First, the sun, in its infinite wisdom, heats up all that water on Earth, transforming it into invisible "water vapor" that floats up like tiny, shy ghosts trying to escape chores. Next, as those vapor ghosts ascend, they cool down, huddling together around minuscule dust particles to form fluffy clouds – basically, a giant, grey sky slumber party. Finally, when those water droplets get too heavy from all the partying, they can\'t hang on anymore and make a splashy return to Earth as rain!\n'),
 Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Alright, buckle up, buttercup, because here\'s the watery t

In [156]:
print('Thinking: \n================\n', response.candidates[0].content.parts[0].text)
print('Response: \n================\n', response.candidates[0].content.parts[1].text)

Thinking: 
 **Understanding Rain: A Lighthearted Explanation**

Okay, here's how I'd explain rain: First, the sun, in its infinite wisdom, heats up all that water on Earth, transforming it into invisible "water vapor" that floats up like tiny, shy ghosts trying to escape chores. Next, as those vapor ghosts ascend, they cool down, huddling together around minuscule dust particles to form fluffy clouds – basically, a giant, grey sky slumber party. Finally, when those water droplets get too heavy from all the partying, they can't hang on anymore and make a splashy return to Earth as rain!

Response: 
 Alright, buckle up, buttercup, because here's the watery truth!

First, the sun acts like a giant hairdryer, warming up water on Earth until it turns into invisible "water vapor" that floats up into the sky like tiny, shy ghosts trying to escape chores. As these vapor ghosts rise higher, they get super chilly, huddle together around microscopic dust particles, and condense into fluffy clouds

Structured Output

In [159]:
class Response(BaseModel):
    question_difficulty: int = Field(description="The difficulty of the question, 1 being the easiest and 10 being the hardest", ge=1, le=10)
    thinking_trace: str = Field(description="Short summary of the thinking process")
    answer: str = Field(description="The answer to the question")
    follow_up_questions: List[str] = Field(description="Follow up questions that are related to the answer")

In [165]:
response = await client.aio.models.generate_content(
    model=model, 
    contents=user_instruction,
    config=types.GenerateContentConfig(
        system_instruction = system_instruction,
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            thinking_budget=4096
            ),
        temperature=0.3,
        candidate_count=1,
        response_mime_type='application/json', #text/plain', #application/json,
        response_schema= Response,
        tools = None,
        tool_config = None,
    )
)

In [166]:
print(f'Prompt tokens: {response.usage_metadata.prompt_token_count}')
print(f'Thoughts tokens: {response.usage_metadata.thoughts_token_count}')
print(f'Candidates tokens: {response.usage_metadata.candidates_token_count}')

print(f'Total tokens: {response.usage_metadata.total_token_count}')

Prompt tokens: 44
Thoughts tokens: 446
Candidates tokens: 261
Total tokens: 751


In [167]:
response.candidates[0].content.parts

[Part(video_metadata=None, thought=True, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='**My Rain-Making Musings**\n\nOkay, so the user wants the lowdown on rain in three sentences, easy and amusing. My thinking started with the core process: evaporation, condensation, then precipitation. I figured, "Let\'s make this fun!"  So, instead of dry scientific terms, I imagined water as a bored ground-dweller who becomes a vapor-ghost at a cloud party, forming tiny liquid droplets. Then, I pictured those droplets as bumper cars, merging until they\'re massive, gravity-defying blobs. Finally, I envisioned gravity as the party-pooper, kicking the now-too-heavy blobs out of the sky as rain.  I think that\'s a good approach to this, hopefully capturing the scientific principles while keeping it light. Now, let\'s craft the JSON response...\n'),
 Part(video_metadata=None, thought=None, inline_data=None, file_dat

In [168]:
print('Thinking: \n================\n', response.candidates[0].content.parts[0].text)
print('Response: \n================\n', response.candidates[0].content.parts[1].text)

Thinking: 
 **My Rain-Making Musings**

Okay, so the user wants the lowdown on rain in three sentences, easy and amusing. My thinking started with the core process: evaporation, condensation, then precipitation. I figured, "Let's make this fun!"  So, instead of dry scientific terms, I imagined water as a bored ground-dweller who becomes a vapor-ghost at a cloud party, forming tiny liquid droplets. Then, I pictured those droplets as bumper cars, merging until they're massive, gravity-defying blobs. Finally, I envisioned gravity as the party-pooper, kicking the now-too-heavy blobs out of the sky as rain.  I think that's a good approach to this, hopefully capturing the scientific principles while keeping it light. Now, let's craft the JSON response...

Response: 
 {
  "question_difficulty": 3,
  "thinking_trace": "I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.

In [179]:
parsed_response = json.loads(response.candidates[0].content.parts[1].text)
parsed_response

{'question_difficulty': 3,
 'thinking_trace': "I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.",
 'answer': 'First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!',
 'follow_up_questions': ['So, clouds are just really crowded water parties?',
  "What's the difference between rain, snow, and hail? Do they just have different dress codes?",
  "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?",
  'Can I make it rain by just st

In [180]:
Response.model_validate(parsed_response)

Response(question_difficulty=3, thinking_trace="I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.", answer='First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!', follow_up_questions=['So, clouds are just really crowded water parties?', "What's the difference between rain, snow, and hail? Do they just have different dress codes?", "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?", 'Can I make it rain by just staring really 

In [181]:
response.parsed

Response(question_difficulty=3, thinking_trace="I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.", answer='First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!', follow_up_questions=['So, clouds are just really crowded water parties?', "What's the difference between rain, snow, and hail? Do they just have different dress codes?", "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?", 'Can I make it rain by just staring really 